In [1]:
import numpy as np
import pandas as pd
import yfinance
from numpy import linalg as LA


from __future__ import division, print_function
from builtins import reversed
from builtins import map, zip
from collections.abc import MutableSequence, Sequence
import copy
from math import ceil
from numbers import Complex, Integral, Real
import sys
import warnings

Copio a continuación las funciones de la librería PyRMT tal cuál están en el github, con algunos comentarios mios pero sin ninguna modificación:

In [2]:
# FUNCIONES AUXILIARES PYRMT

def checkDesignMatrix(X):
    """
       Parameters
       ----------
       X: a matrix of shape (T, N), where T denotes the number
           of samples and N labels the number of features.
           If T < N, a warning is issued to the user, and the transpose
           of X is considered instead.
       Returns:
       T: type int
       N: type int
       transpose_flag: type bool
           Specify if the design matrix X should be transposed
           in view of having less rows than columns.       
    """
    
    try:
        assert isinstance(X, (np.ndarray, pd.DataFrame, pd.Series))
    except AssertionError:
        raise
        sys.exit(1)

    # Código para convertir a X en un array, si es que no lo es  
    X = np.asarray(X, dtype=float)
    # Código para convertir a X en un array de numpy de dos dimensiones 
    X = np.atleast_2d(X)

    if X.shape[0] < X.shape[1]:
        warnings.warn("The Marcenko-Pastur distribution pertains to "
                      "the empirical covariance matrix of a random matrix X "
                      "of shape (T, N). It is assumed that the number of "
                      "samples T is assumed higher than the number of "
                      "features N. The transpose of the matrix X submitted "
                      "at input will be considered in the cleaning schemes "
                      "for the corresponding correlation matrix.", UserWarning)
        
        T, N = reversed(X.shape)
        transpose_flag = True
    else:
        T, N = X.shape
        transpose_flag = False
        
    return T, N, transpose_flag

def xiHelper(x, q, E):
    """Helper function to the rotationally-invariant, optimal shrinkage
       estimator of the true correlation matrix (implemented via function
       optimalShrinkage of the present module). 
       Parameters
       ----------
       x: type derived from numbers.Real
           Would typically be expected to be an eigenvalue from the
           spectrum of correlation matrix E. The present function
           can however handle an arbitrary floating-point number.
       q: type derived from numbers.Real
           The number parametrizing a Marcenko-Pastur spectrum.
       E: type numpy.ndarray
           Symmetric correlation matrix associated with the 
           Marcenko-Pastur parameter q specified above.
       Returns
       -------
       xi: type float
           Cleaned eigenvalue of the true correlation matrix C underlying
           the empirical correlation E (the latter being corrupted 
           with in-sample noise). This cleaned version is computed
           assuming no prior knowledge on the structure of the true
           eigenvectors (thereby leaving the eigenvectors of E unscathed). 
       References
       ----------
       * "Rotational invariant estimator for general noisy matrices",
         J. Bun, R. Allez, J.-P. Bouchaud and M. Potters
         arXiv: 1502.06736 [cond-mat.stat-mech]
       * "Cleaning large Correlation Matrices: tools from Random Matrix Theory",
         J. Bun, J.-P. Bouchaud and M. Potters
         arXiv: 1610.08104 [cond-mat.stat-mech]
    """

    try:
        assert isinstance(x, Real)
        assert isinstance(q, Real)
        assert isinstance(E, np.ndarray) and E.shape[0] == E.shape[1]
        assert np.allclose(E.transpose(1, 0), E)
    except AssertionError:
        raise
        sys.exit(1)

    N = E.shape[0]
    
    # esta z es z_k en el paper Box1 (ESTÁ BIEN)
    z = x - 1j / np.sqrt(N)
    
    # esta s_k es en realidad s_k(z_k) en el paper (ESTA DIFERENTE)
    s = stieltjes(z, E)
    
    #xi es \xi_l^{RIE} en el paper (ESTA IGUAL)
    xi = x / abs(1 - q + q * z * s)**2
   
    return xi

def stieltjes(z, E):
    """
       Parameters
       ----------
       z: complex number
       E: square matrix
       Returns
       -------
       A complex number, the resolvent of square matrix E, 
       also known as its Stieltjes transform.
       Reference
       ---------
       "Financial Applications of Random Matrix Theory: a short review",
       J.-P. Bouchaud and M. Potters
       arXiv: 0910.1205 [q-fin.ST]
    """

    try:
        assert isinstance(z, Complex)
        
        assert isinstance(E, (np.ndarray, pd.DataFrame))
        E = np.asarray(E, dtype=float)
        E = np.atleast_2d(E)
        assert E.shape[0] == E.shape[1]
    except AssertionError:
        raise
        sys.exit(1)

    #AQUÍ ES DONDE SE ESTÁ CALCULANDO DE FORMA DISTINTA LO QUE SE CONOCE COMO $S_k$ EN EL PAPER
    # Y AQUÍ SE LE DENOMINA ret
    N = E.shape[0]
    
    ret = z * np.eye(N, dtype=float) - E
    
   
    ret = np.trace(ret) / N
   

    return ret

def gammaHelper(x, q, N, lambda_N, inverse_wishart=False):
    """Helper function to optimalShrinkage function defined below.
       The eigenvalue to the cleaned estimator of a true correlation
       matrix are computed via the function xiHelper defined above in
       the module at hand. 
       
       It is known however that when N is not very large
       a systematic downward bias affects the xiHelper estimator for small
       eigenvalues of the noisy empirical correlation matrix. This bias
       can be heuristically corrected by computing
       xi_hat = xi_RIE * max(1, Gamma),
       with Gamma evaluated by the function gammaHelper herewith.
       Parameters
       ----------
       x: type float or any other type derived from numbers.Real
           Typically an eigenvalue from the spectrum of a sample
           estimate of the correlation matrix associated to some
           design matrix X. However, the present function supports
           any arbitrary floating-point number x at input.
       q: type derived from numbers.Real
           Parametrizes a Marcenko-Pastur spectrum.
       N: type derived from numbers.Integral
           Dimension of a correlation matrix whose debiased, 
           rotationally-invariant estimator is to be assessed via
           the function RIE (see below), of which the present function
           is a helper.
       lambda_N: type derived from numbers.Real
           Smallest eigenvalue from the spectrum of an empirical
           estimate to a correlation matrix.
        
       inverse_wishart: type bool default: False
            Wether to use inverse wishart regularization
       Returns
       ------
       Gamma: type float
           Upward correction factor for computing a debiased 
           rotationally-invariant estimator of a true underlying 
           correlation matrix. 
       Reference
       ---------
       "Cleaning large Correlation Matrices: tools from Random Matrix Theory",
        J. Bun, J.-P. Bouchaud and M. Potters
        arXiv: 1610.08104 [cond-mat.stat-mech]
    """

    try:
        assert isinstance(x, Real)
        assert isinstance(q, Real)
        assert isinstance(N, Integral)
        assert isinstance(lambda_N, Real)
    except AssertionError:
        raise
        sys.exit(1)

    z = x - 1j / np.sqrt(N)
  
    # Este lambda plus está igual  que en el paper (ESTÁ BIEN)
    lambda_plus = (1 + np.sqrt(q))**2

    lambda_plus /= (1 - np.sqrt(q))**2

    lambda_plus *= lambda_N

    # Este sigma cuadrada está igual que en el N (ESTÁ BIEN)
    sigma_2 = lambda_N / (1 - np.sqrt(q))**2

    
    # gmp defined below stands for the Stieltjes transform of the
    # rescaled Marcenko-Pastur density, evaluated at z
    # esta función es la (18) y es distinta a la del paper ( NOTAR QUE EN EL PAPER 
    # LA PARTE QUE SE RESTA ESTÁ COMO \SQRT{ z - \lambda_n} \times \sqrt{z - \lambda_+})
    # mientras que aquí primero se multiplican los dos valores y luego se saca la raíz cuadrada
    gmp = z + sigma_2 * (q - 1) - np.sqrt((z - lambda_N) * (z - lambda_plus))    
    gmp /= 2 * q * sigma_2 * z
    # Este es el Gamma_k y ESTÁ IGUAL (más adelante se realiza la división entre lambda_k que está como x)
    Gamma = abs(1 - q + q * z * gmp)**2
    Gamma *= sigma_2
    
    if inverse_wishart:
        kappa = 2 * lambda_N / ((1 - q - lambda_N) ** 2 - 4 * q * lambda_N)
        alpha_s = 1 / (1 + 2 * q * kappa)
        denom = x / (1 + alpha_s * (x - 1.))
        Gamma /= denom
    else: 
        Gamma /= x

    return Gamma


# FUNCIÓN PRINCIPAL PYRMT


In [3]:
def optimalShrinkage(X, return_covariance=False, method='rie'):
    """This function computes a cleaned, optimal shrinkage, 
       rotationally-invariant estimator (RIE) of the true correlation 
       matrix C underlying the noisy, in-sample estimate 
       E = 1/T X * transpose(X)
       associated to a design matrix X of shape (T, N) (T measurements 
       and N features).
       One approach to getting a cleaned estimator that predates the
       optimal shrinkage, RIE estimator consists in inverting the 
       Marcenko-Pastur equation so as to replace the eigenvalues
       from the spectrum of E by an estimation of the true ones.
       This approach is known to be numerically-unstable, in addition
       to failing to account for the overlap between the sample eigenvectors
       and the true eigenvectors. How to compute such overlaps was first
       explained by Ledoit and Peche (cf. reference below). Their procedure
       was extended by Bun, Bouchaud and Potters, who also correct
       for a systematic downward bias in small eigenvalues.
       
       It is this debiased, optimal shrinkage, rotationally-invariant
       estimator that the function at hand implements.
       
       In addition to above method, this funtion also provides access to:  
       - The finite N regularization of the optimal RIE for small eigenvalues
         as provided in section 8.1 of [3] a.k.a the inverse wishart (IW) regularization.
       - The direct kernel method of O. Ledoit and M. Wolf in their 2017 paper [4]. 
         This is a direct port of their Matlab code.
        
         
       Parameters
       ----------
       X: design matrix, of shape (T, N), where T denotes the number
           of samples (think measurements in a time series), while N
           stands for the number of features (think of stock tickers).
           
       return_covariance: type bool (default: False)
           If set to True, compute the standard deviations of each individual
           feature across observations, clean the underlying matrix
           of pairwise correlations, then re-apply the standard
           deviations and return a cleaned variance-covariance matrix.
       
       method: type string, optional (default="rie")
           - If "rie" : optimal shrinkage in the manner of Bun & al.
            with no regularisation  
           - If "iw" : optimal shrinkage in the manner of Bun & al.
            with the so called Inverse Wishart regularization
           - If 'kernel': Direct kernel method of Ledoit  Wolf.
       Returns
       -------
       E_RIE: type numpy.ndarray, shape (N, N)
           Cleaned estimator of the true correlation matrix C. A sample
           estimator of C is the empirical covariance matrix E 
           estimated from X. E is corrupted by in-sample noise.
           E_RIE is the optimal shrinkage, rotationally-invariant estimator 
           (RIE) of C computed following the procedure of Joel Bun 
           and colleagues (cf. references below).
           
           If return_covariance=True, E_clipped corresponds to a cleaned
           variance-covariance matrix.
       References
       ----------
       1 "Eigenvectors of some large sample covariance matrix ensembles",
         O. Ledoit and S. Peche
         Probability Theory and Related Fields, Vol. 151 (1), pp 233-264
       2 "Rotational invariant estimator for general noisy matrices",
         J. Bun, R. Allez, J.-P. Bouchaud and M. Potters
         arXiv: 1502.06736 [cond-mat.stat-mech]
       3 "Cleaning large Correlation Matrices: tools from Random Matrix Theory",
         J. Bun, J.-P. Bouchaud and M. Potters
         arXiv: 1610.08104 [cond-mat.stat-mech]
       4 "Direct Nonlinear Shrinkage Estimation of Large-Dimensional Covariance Matrices (September 2017)", 
         O. Ledoit and M. Wolf https://ssrn.com/abstract=3047302 or http://dx.doi.org/10.2139/ssrn.3047302
    """
    
    try:
        assert isinstance(return_covariance, bool)
    except AssertionError:
        raise
        sys.exit(1)

    T, N, transpose_flag = checkDesignMatrix(X)
    if transpose_flag:
        X = X.T
      
    # Si se quiere la matriz de correlación limpia, los datos SÍ se estandarizan, de lo contrario no se estandarizan
    if not return_covariance:
      print("sí estandarize")
      X = StandardScaler(with_mean=False,with_std=True).fit_transform(X)

    #ec = EmpiricalCovariance(store_precision=False,assume_centered=True)
    #ec.fit(X)
    # COMO YO USO NUMPY PARA OBTENER LA MATRIZ DE CORRELAICÓN MÁS RÁPIDO, HAGO ESTE PEQUEÑO CAMBIO AQUÍ
    # PARA CACULAR LA MATRIZ CON NUMPY Y NO CON SCIKIT LEARN
    #E = ec.covariance_
    E = np.corrcoef(X.T)
    

    if return_covariance:
        inverse_std = 1./np.sqrt(np.diag(E))
        E *= inverse_std
        E *= inverse_std.reshape(-1, 1)

    eigvals, eigvecs = np.linalg.eigh(E)
    eigvecs = eigvecs.T
    eigen_val_sample, eigen_vec_sample = LA.eig(E)
    q = N / float(T)
    lambda_N = eigvals[0]  # The smallest empirical eigenvalue,
                           # given that the function used to compute
                           # the spectrum of a Hermitian or symmetric
                           # matrix - namely np.linalg.eigh - returns
                           # the eigenvalues in ascending order.
    lambda_hats = None

    
    if method is not 'kernel':
        use_inverse_wishart = (method == 'iw')
        xis = map(lambda x: xiHelper(x, q, E), eigvals)
        Gammas = map(lambda x: gammaHelper(x, q, N, lambda_N, inverse_wishart=use_inverse_wishart), eigvals)
        # Aquí se define el stepwise function ( y está igual que en el paper)
        xi_hats = map(lambda a, b: a * b if b > 1 else a, xis, Gammas)
        lambda_hats = xi_hats
    else:
         lambda_hats = directKernel(q, T, N, eigvals)
        
    E_RIE = np.zeros((N, N), dtype=float)
    
    for lambda_hat, eigvec in zip(lambda_hats, eigvecs):
        
        eigvec = eigvec.reshape(-1, 1)
        E_RIE += lambda_hat * eigvec.dot(eigvec.T)
        
    # EN MI IMPLEMENTACIÓN YA NO MODIFICO LA MATRIZ PARA QUE LA DIAGONAL SEA EXACTAMENTE UNO
    # QUE ES LO QUE SE HACE ADELANTE, RAZÓN POR LA CUAL IMPRIMO LA MATRIZ EN ESTE PUNTO ANTES 
    # DE QUE LA MODIFIQUEN PARA VERIFICAR QUE LOS RESULTADOS SEAN LOS MISMOS
    print(E_RIE)
    print(LA.eig)
    tmp = 1./np.sqrt(np.diag(E_RIE))
    E_RIE *= tmp
    E_RIE *= tmp.reshape(-1, 1)
    
    if return_covariance:
        std = 1./inverse_std
        E_RIE *= std
        E_RIE *= std.reshape(-1, 1)
    

    return E_RIE

<>:114: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:114: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_5619/842616512.py:114: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if method is not 'kernel':


En la implementación de PyRMT hay dos cosas que cambian con respecto al paper:
 - El cálculo de $S_k(z_k)$
 - El cálculo de $g_{mp}(z)$

A continuación escribo mi implementación calculando estas dos variables tal cuál se hace en el PyRMT para mostrar que son los únicos dos lugares donde la implementación difiere, y que se utiliza la misma forma de cálculo se obtienen entonces los mismos resultados

In [4]:
# FUNCION AUXILIAR MI IMPLEMENTACION

def returnsStandardization(returns):
    returns_wth_mean = returns - np.mean(returns, axis=0)
    hat_sigma = np.sqrt((returns_wth_mean**2).sum(axis=1))
    r_tilde = returns_wth_mean.divide(hat_sigma, axis=0)
    X = r_tilde / np.std(r_tilde)
    return X


# FUNCION PRINCIPAL MI IMPLEMENTACION


def get_rie(returns, normalize=False):
    def get_s_k(index_lambda, N):
        return 1/N * (sum(1/(z_k[index_lambda] - lambdas)) - 1/(z_k[index_lambda] - lambdas[index_lambda]))

    # T is the number of observations, N is the number of assets
    T, N = returns.shape
    RIE_estimator = np.zeros((N, N), dtype=float)
    if normalize:
        returns = returnsStandardization(returns)
    # Calculation of the sample correlation matrix
    E = np.corrcoef(returns.T)
    # The eigenvalues and eigenvectors of the returns are obtained
    lambdas, u_ks = LA.eigh(E)
    u_ks = u_ks.T
    n_lambda = lambdas[0]
    q = float(N/T)
    sigma_sq = (n_lambda)/(1 - np.sqrt(q))**2
    lambda_plus = n_lambda*((1+np.sqrt(q))/(1 - np.sqrt(q)))**2
    # Get z_k
    z_k = lambdas - (1j / np.sqrt(N))
    # Get s_k(z_k)
    s_k = z_k - 1
    #s_k = list(map(lambda index_lambda: get_s_k(
    #    index_lambda, N), np.argsort(lambdas)))
    # Get \xi_k^{RIE}
    xi_k = lambdas / np.abs(1 - q + q * z_k * s_k)**2
    # Get stieltjes g_{mp}(z)
    #g_mp = (z_k + sigma_sq*(q-1) - (np.sqrt(z_k - n_lambda)
    #        * np.sqrt(z_k - lambda_plus)))/(2*q*z_k*sigma_sq)
    g_mp = (z_k + sigma_sq*(q-1) - (np.sqrt((z_k - n_lambda)* (z_k - lambda_plus))))/(2*q*z_k*sigma_sq)
    # Get gamma_k(z_k)
    gamma_k = sigma_sq * ((np.abs(1 - q + q*z_k*g_mp)**2)/(lambdas))
    # Get \hat{xh}_k
    xi_hat = list(map(lambda xi, gamma: xi *
                  gamma if gamma > 1 else xi, xi_k, gamma_k))
    # Get RIE
    for xi, u_i in zip(xi_hat, u_ks):
        RIE_estimator += xi*(u_i.reshape(-1, 1) @ u_i.reshape(-1, 1).T)

    return RIE_estimator

In [5]:
test_set = yfinance.download(['FB', 'AOS', 'ABT', 'GOOGL'], start="2020-01-01", end = "2021-12-31")
returns = test_set['Close'].pct_change(periods=1).dropna()


[*********************100%***********************]  4 of 4 completed


In [6]:
returns

,ABT,AOS,FB,GOOGL
Date,,,,
2020-01-03,-0.012191,-0.008792,-0.005291,-0.005231
2020-01-06,0.005239,0.006336,0.018834,0.026654
2020-01-07,-0.005559,-0.006716,0.002164,-0.001932
2020-01-08,0.004076,-0.001479,0.010138,0.007118
2020-01-09,0.002668,-0.004443,0.014311,0.010498
...,...,...,...,...
2021-12-23,0.001223,0.014338,0.014495,0.003425
2021-12-27,0.016528,0.023318,0.032633,0.006738
2021-12-28,-0.006998,0.009917,0.000116,-0.008245


Estandarizo los retornos como se describe en el artículo (a las dos funciones se les pasara este input)

In [7]:
returns_estand = returnsStandardization(returns)

A continuación calculo el RIE mediante mi implementación pero con la misma forma que usa PyRMT para calcular $S_k(z_k)$ y $g_{mp}(z)$

In [8]:
print(get_rie(returns_estand, False))

[[1.02519059 0.09057524 0.18712667 0.20985177]
 [0.09057524 1.01883383 0.11750129 0.13346603]
 [0.18712667 0.11750129 1.17695412 0.33697996]
 [0.20985177 0.13346603 0.33697996 1.21128441]]


A continuación utilizo la implementación de PyRMT. La primer matriz que se muestra es el RIE antes de modificar las entradas para que se obtengan unos en la diagonal. **Es posible verificar entonces que el resultado es el mismo**.

In [22]:
optimalShrinkage(returns_estand, True)

[[1.02519059 0.09057524 0.18712667 0.20985177]
 [0.09057524 1.01883383 0.11750129 0.13346603]
 [0.18712667 0.11750129 1.17695412 0.33697996]
 [0.20985177 0.13346603 0.33697996 1.21128441]]
<function eig at 0x7f6d4c1d9120>


array([[1.        , 0.08862484, 0.17035451, 0.18831602],
       [0.08862484, 1.        , 0.10730284, 0.12014232],
       [0.17035451, 0.10730284, 1.        , 0.28222875],
       [0.18831602, 0.12014232, 0.28222875, 1.        ]])

Se vuelve a definir la función pero utilizando la forma que indica el paper para calcular  $S_k(z_k)$ y $g_{mp}(z)$ (Se ponen comentadas la forma que usa PyRMt para comparación):

In [23]:
# FUNCION AUXILIAR MI IMPLEMENTACION

def returnsStandardization(returns):
    returns_wth_mean = returns - np.mean(returns, axis=0)
    hat_sigma = np.sqrt((returns_wth_mean**2).sum(axis=1))
    r_tilde = returns_wth_mean.divide(hat_sigma, axis=0)
    X = r_tilde / np.std(r_tilde)
    return X


# FUNCION PRINCIPAL MI IMPLEMENTACION


def get_rie(returns, normalize=False):
    def get_s_k(index_lambda, N):
        return 1/N * (sum(1/(z_k[index_lambda] - lambdas)) - 1/(z_k[index_lambda] - lambdas[index_lambda]))

    # T is the number of observations, N is the number of assets
    T, N = returns.shape
    RIE_estimator = np.zeros((N, N), dtype=float)
    if normalize:
        returns = returnsStandardization(returns)
    # Calculation of the sample correlation matrix
    E = np.corrcoef(returns.T)
    # The eigenvalues and eigenvectors of the returns are obtained
    lambdas, u_ks = LA.eigh(E)
    u_ks = u_ks.T
    n_lambda = lambdas[0]
    q = float(N/T)
    sigma_sq = (n_lambda)/(1 - np.sqrt(q))**2
    lambda_plus = n_lambda*((1+np.sqrt(q))/(1 - np.sqrt(q)))**2
    # Get z_k
    z_k = lambdas - (1j / np.sqrt(N))
    # Get s_k(z_k)
    #s_k = z_k - 1
    s_k = list(map(lambda index_lambda: get_s_k(
        index_lambda, N), np.argsort(lambdas)))
    # Get \xi_k^{RIE}
    xi_k = lambdas / np.abs(1 - q + q * z_k * s_k)**2
    # Get stieltjes g_{mp}(z)
    g_mp = (z_k + sigma_sq*(q-1) - (np.sqrt(z_k - n_lambda)
            * np.sqrt(z_k - lambda_plus)))/(2*q*z_k*sigma_sq)
    #g_mp = (z_k + sigma_sq*(q-1) - (np.sqrt((z_k - n_lambda)* (z_k - lambda_plus))))/(2*q*z_k*sigma_sq)
    # Get gamma_k(z_k)
    gamma_k = sigma_sq * ((np.abs(1 - q + q*z_k*g_mp)**2)/(lambdas))
    # Get \hat{xh}_k
    xi_hat = list(map(lambda xi, gamma: xi *
                  gamma if gamma > 1 else xi, xi_k, gamma_k))
    # Get RIE
    for xi, u_i in zip(xi_hat, u_ks):
        RIE_estimator += xi*(u_i.reshape(-1, 1) @ u_i.reshape(-1, 1).T)

    return RIE_estimator

Vuelvo a correr la función con los mismos datos, **es posible observar que el resultado es diferente**

In [24]:
get_rie(returns_estand, False)

array([[1.00799956, 0.08946465, 0.15965291, 0.24226455],
       [0.08946465, 1.00617941, 0.11209109, 0.14159726],
       [0.15965291, 0.11209109, 1.04416625, 0.47395155],
       [0.24226455, 0.14159726, 0.47395155, 1.05236458]])